In [8]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

DATA_DIR = r"archive"  
IMG_SIZE = 96
BATCH_SIZE = 64
SEED = 42
EPOCHS = 25


In [9]:
train_dir = os.path.join(DATA_DIR, "train")
test_dir  = os.path.join(DATA_DIR, "test")

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)
print("Num classes:", num_classes)


Found 16108 files belonging to 8 classes.
Using 12887 files for training.
Found 16108 files belonging to 8 classes.
Using 3221 files for validation.
Found 14518 files belonging to 8 classes.
Classes: ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Num classes: 8


In [10]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)


In [11]:
data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.10),
    layers.RandomContrast(0.10),
], name="augmentation")


In [12]:
base = tf.keras.applications.EfficientNetB0(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_aug(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 augmentation (Sequential)   (None, 96, 96, 3)         0         
                                                                 
 efficientnetb0 (Functional  (None, 3, 3, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 8)                 10248 

In [13]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint("affectnet_best.keras", save_best_only=True, monitor="val_accuracy", mode="max"),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss")
]


In [14]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=2
)


Epoch 1/25




KeyboardInterrupt: 

In [8]:
base.trainable = True

fine_tune_at = int(len(base.layers) * 0.5)  # فك 50% الأخيرة بدل 
for layer in base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(3e-5),  # أصغر من 1e-4 لتحافظ على الميزات
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,           # أكثر شوي
    callbacks=callbacks,
    verbose=2
)


Epoch 1/15
202/202 - 119s - loss: 1.4029 - accuracy: 0.4718 - val_loss: 1.3485 - val_accuracy: 0.4859 - lr: 3.0000e-05 - 119s/epoch - 590ms/step
Epoch 2/15
202/202 - 108s - loss: 1.2816 - accuracy: 0.5173 - val_loss: 1.2705 - val_accuracy: 0.5172 - lr: 3.0000e-05 - 108s/epoch - 534ms/step
Epoch 3/15
202/202 - 118s - loss: 1.2136 - accuracy: 0.5435 - val_loss: 1.2182 - val_accuracy: 0.5359 - lr: 3.0000e-05 - 118s/epoch - 585ms/step
Epoch 4/15
202/202 - 110s - loss: 1.1678 - accuracy: 0.5569 - val_loss: 1.1856 - val_accuracy: 0.5477 - lr: 3.0000e-05 - 110s/epoch - 543ms/step
Epoch 5/15
202/202 - 107s - loss: 1.1189 - accuracy: 0.5772 - val_loss: 1.1577 - val_accuracy: 0.5591 - lr: 3.0000e-05 - 107s/epoch - 530ms/step
Epoch 6/15
202/202 - 102s - loss: 1.0690 - accuracy: 0.5991 - val_loss: 1.1383 - val_accuracy: 0.5666 - lr: 3.0000e-05 - 102s/epoch - 504ms/step
Epoch 7/15
202/202 - 101s - loss: 1.0367 - accuracy: 0.6113 - val_loss: 1.1219 - val_accuracy: 0.5725 - lr: 3.0000e-05 - 101s/epoc

In [15]:
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)


227/227 - 64s - loss: 2.1410 - accuracy: 0.1390 - 64s/epoch - 284ms/step
Test loss: 2.140988826751709
Test accuracy: 0.13899986445903778


In [16]:
model.save("affectnet_final.keras")
print("Saved: affectnet_final.keras")

with open("class_names.txt", "w", encoding="utf-8") as f:
    for c in class_names:
        f.write(c + "\n")

print("Saved: class_names.txt")


Saved: affectnet_final.keras
Saved: class_names.txt


In [6]:
import cv2
import numpy as np
from tensorflow import keras
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

# ==================================================
#                 الإعدادات العامة
# ==================================================
MODEL_PATH = "affectnet_final.keras"
CLASS_NAMES_PATH = "class_names.txt"
IMG_SIZE = 96

# ==================================================
#         تحميل نموذج التعلم العميق
# ==================================================
model = keras.models.load_model(MODEL_PATH)

with open(CLASS_NAMES_PATH, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

# ==================================================
#              إعداد واجهة Tkinter
# ==================================================
root = tk.Tk()
root.title("AI Emotion Detection System")
root.geometry("1000x650")
root.configure(bg="#1e1e1e")

# ===== عنوان =====
title = tk.Label(root, text="Real-Time Emotion Detection",
                 font=("Arial", 20, "bold"),
                 fg="#00d4ff", bg="#1e1e1e")
title.pack(pady=10)

# ===== إطار الفيديو =====
video_frame = tk.Frame(root, bg="#1e1e1e")
video_frame.pack()

video_label = tk.Label(video_frame)
video_label.pack()

# ==================================================
#              لوحة التحكم
# ==================================================
control_frame = tk.Frame(root, bg="#2b2b2b")
control_frame.pack(fill="x", pady=10)

brightness_var = tk.IntVar(value=50)
blur_var = tk.IntVar(value=1)
threshold_var = tk.IntVar(value=0)
emotion_var = tk.BooleanVar(value=True)

def styled_scale(label_text, variable, from_, to_):
    frame = tk.Frame(control_frame, bg="#2b2b2b")
    frame.pack(side="left", padx=20)
    tk.Label(frame, text=label_text, fg="white",
             bg="#2b2b2b").pack()
    scale = ttk.Scale(frame, from_=from_, to=to_,
                      variable=variable, orient="horizontal", length=150)
    scale.pack()

styled_scale("Brightness", brightness_var, 0, 100)
styled_scale("Blur", blur_var, 0, 20)
styled_scale("Threshold", threshold_var, 0, 255)

# ==================================================
#                 وظائف الأزرار
# ==================================================
def capture_image():
    ret, frame = cap.read()
    if ret:
        cv2.imwrite("captured_image.png", frame)
        print("تم حفظ الصورة ✅")

def reset_values():
    brightness_var.set(50)
    blur_var.set(1)
    threshold_var.set(0)

def toggle_emotion():
    emotion_var.set(not emotion_var.get())

def exit_app():
    cap.release()
    root.destroy()

button_frame = tk.Frame(root, bg="#1e1e1e")
button_frame.pack(pady=10)

tk.Button(button_frame, text="📸 Capture",
          command=capture_image,
          bg="#4CAF50", fg="white",
          width=12).pack(side="left", padx=10)

tk.Button(button_frame, text="🔄 Reset",
          command=reset_values,
          bg="#ff9800", fg="white",
          width=12).pack(side="left", padx=10)

tk.Button(button_frame, text="🧠 Emotion ON/OFF",
          command=toggle_emotion,
          bg="#2196F3", fg="white",
          width=15).pack(side="left", padx=10)

tk.Button(button_frame, text="❌ Exit",
          command=exit_app,
          bg="#f44336", fg="white",
          width=12).pack(side="left", padx=10)

# ==================================================
#              معالجة الفيديو
# ==================================================
def update_frame():
    ret, frame = cap.read()
    if not ret:
        return

    processed = frame.copy()

    # ===== تحسين إضاءة تلقائي (CLAHE) =====
    lab = cv2.cvtColor(processed, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    processed = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # ===== إضاءة يدوية =====
    beta = brightness_var.get() - 50
    processed = cv2.convertScaleAbs(processed, alpha=1.0, beta=beta)

    # ===== Blur =====
    blur_value = int(blur_var.get())
    if blur_value > 0:
        if blur_value % 2 == 0:
            blur_value += 1
        processed = cv2.GaussianBlur(processed, (blur_value, blur_value), 0)

    # ===== Threshold =====
    if threshold_var.get() > 0:
        gray_temp = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
        _, thresh_img = cv2.threshold(
            gray_temp, threshold_var.get(), 255, cv2.THRESH_BINARY)
        processed = cv2.cvtColor(thresh_img, cv2.COLOR_GRAY2BGR)

    # ===== كشف الوجه =====
    gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)

    if emotion_var.get():
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            roi = processed[y:y+h, x:x+w]
            cv2.rectangle(processed, (x, y),
                          (x+w, y+h), (0,255,0), 2)

            face_resized = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))
            face_array = face_resized.astype("float32") / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            prediction = model.predict(face_array, verbose=0)[0]
            best_index = np.argmax(prediction)
            emotion = class_names[best_index]
            confidence = prediction[best_index] * 100

            cv2.putText(processed,
                        f"{emotion} ({confidence:.1f}%)",
                        (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7, (0,255,0), 2)

    # ===== عرض داخل Tk =====
    img_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img_rgb)
    imgtk = ImageTk.PhotoImage(image=img)

    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    root.after(10, update_frame)

update_frame()
root.mainloop()


In [13]:
import tensorflow as tf

MODEL_PATH = "affectnet_final.keras"
OUT_PATH = "emotion_model.tflite"

model = tf.keras.models.load_model(MODEL_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)


tflite_model = converter.convert()

with open(OUT_PATH, "wb") as f:
    f.write(tflite_model)

print("Saved:", OUT_PATH)


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmp_a6jpfus\assets


INFO:tensorflow:Assets written to: C:\Users\USER\AppData\Local\Temp\tmp_a6jpfus\assets


Saved: emotion_model.tflite
